In [23]:
! pip install keras-tuner -q

In [24]:
import keras
from keras import layers

# HOW TO MAKE NUMBER OF NEURONS AS HYPERPARAMETRE

In [25]:
def build_model(hp):
    model=keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("units",min_value=32,max_value=512,step=32),
            activation='relu'
        )
    )
    model.add(layers.Dense(10,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

# HOW TO MAKE OTHER PARAMETERS AS HYPERPARAMETRE

In [26]:
def build_model(hp):
    model=keras.Sequential()
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=hp.Int("units",min_value=32,max_value=512,step=32),
            activation=hp.Choice('activation',['relu','tanh'])
        )
    )
    if hp.Boolean('dropout'):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10,activation='softmax'))
    learning_rate=hp.Float('lr',min_value=1e-4,max_value=1e-2,sampling='log')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

# HOW TO MAKE NUMBER OF LAYERS AS HYPERPARAMETRE


In [27]:
def build_model(hp):
    model=keras.Sequential()
    model.add(layers.Flatten())
    for i in range(hp.Int('num_layers',1,3)):
        
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}",min_value=32,max_value=512,step=32),
                activation=hp.Choice('activation',['relu','tanh'])
            )
        )
    if hp.Boolean('dropout'):
        model.add(layers.Dropout(rate=0.25))
    if hp.Boolean('batch_norm'):
        model.add(layers.BatchNormalization())
    model.add(layers.Dense(10,activation='softmax'))
    learning_rate=hp.Float('lr',min_value=1e-4,max_value=1e-2,sampling='log')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [28]:
import keras_tuner
tuner=keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory='my_dir',
    project_name='helloworld'
)

In [29]:
import keras
import numpy as np

In [30]:
(x,y),(x_test,y_test)=keras.datasets.mnist.load_data()

In [31]:
x_train=x[:-10000]
y_train=y[:-10000]
x_val=x[-10000:]
y_val=y[-10000:]

In [32]:
x_train=np.expand_dims(x_train,-1).astype('float32')/255.0
x_val=np.expand_dims(x_val,-1).astype('float32')/255.0
x_test=np.expand_dims(x_test,-1).astype('float32')/255.0

In [33]:
num_classes=10
y_train=keras.utils.to_categorical(y_train,num_classes)
y_val=keras.utils.to_categorical(y_val,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [34]:
tuner.search(x_train,y_train,epochs=2,validation_data=(x_val,y_val))

Trial 3 Complete [00h 00m 51s]
val_accuracy: 0.967199981212616

Best val_accuracy So Far: 0.967199981212616
Total elapsed time: 00h 05m 17s


In [35]:
models=tuner.get_best_models(num_models=2)
best_model=models[0]

C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [36]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 96)             │        75,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           970 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,714 (299.66 KB)

 Trainable params: 76,522 (298.91 KB)

 Non-trainable params: 192 (768.00 B)

In [40]:
print(tuner.results_summary())

Results summary
Results in my_dir\helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 1
units_0: 96
activation: tanh
dropout: False
batch_norm: True
lr: 0.002949855433472653
units_1: 512
units_2: 384
Score: 0.967199981212616

Trial 1 summary
Hyperparameters:
num_layers: 3
units_0: 128
activation: tanh
dropout: False
batch_norm: True
lr: 0.0006805637517182109
units_1: 96
units_2: 32
Score: 0.9644500017166138

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 448
activation: relu
dropout: True
batch_norm: False
lr: 0.006944763609999221
units_1: 32
Score: 0.9610500037670135
None
